# Batch  processing

## Examples

In [1]:
import findspark
findspark.init()
import pyspark
import random

from pyspark.sql import SparkSession

sc = pyspark.SparkContext(appName="SPARK_API")
spark = SparkSession(sc)


In [2]:
# Number of passengers in each class in the Titanic dataset:

from pyspark.sql.functions import count

df = spark.read.option("header", "true").option("inferSchema", "true").csv("titanic.csv")

passengerCounts = df.groupBy("Pclass").agg(count("*").alias("count"))

passengerCounts.show()



+------+-----+
|Pclass|count|
+------+-----+
|     1|  216|
|     3|  491|
|     2|  184|
+------+-----+



In [3]:
# Determine the number of passengers by age group (e.g. 0-10 years, 11-20 years, etc.) and by class:

from pyspark.sql.functions import count, when

df = spark.read.option("header", "true").option("inferSchema", "true").csv("titanic.csv")

ageRanges = [(0, 10), (11, 20), (21, 30), (31, 40), (41, 50), (51, 60), (61, 70), (71, 80)]

ageRangesDf = df
for lower, upper in ageRanges:
    ageRangesDf = ageRangesDf.withColumn(f"AgeRange_{lower}_to_{upper}", when(df.Age.between(lower, upper), 1).otherwise(0))

passengerCountsByAge = ageRangesDf.groupBy("Pclass") \
                                 .agg(count(when(ageRangesDf.AgeRange_0_to_10 == 1, True)).alias("0-10"),
                                      count(when(ageRangesDf.AgeRange_11_to_20 == 1, True)).alias("11-20"),
                                      count(when(ageRangesDf.AgeRange_21_to_30 == 1, True)).alias("21-30"),
                                      count(when(ageRangesDf.AgeRange_31_to_40 == 1, True)).alias("31-40"),
                                      count(when(ageRangesDf.AgeRange_41_to_50 == 1, True)).alias("41-50"),
                                      count(when(ageRangesDf.AgeRange_51_to_60 == 1, True)).alias("51-60"),
                                      count(when(ageRangesDf.AgeRange_61_to_70 == 1, True)).alias("61-70"),
                                      count(when(ageRangesDf.AgeRange_71_to_80 == 1, True)).alias("71-80"))

passengerCountsByAge.show()


+------+----+-----+-----+-----+-----+-----+-----+-----+
|Pclass|0-10|11-20|21-30|31-40|41-50|51-60|61-70|71-80|
+------+----+-----+-----+-----+-----+-----+-----+-----+
|     1|   3|   18|   40|   49|   37|   25|   11|    3|
|     3|  44|   79|  128|   61|   28|    5|    3|    1|
|     2|  17|   18|   61|   43|   19|   12|    3|    0|
+------+----+-----+-----+-----+-----+-----+-----+-----+



In [4]:
# Number of passengers by gender in each class

from pyspark.sql.functions import count

df = spark.read.option("header", "true").option("inferSchema", "true").csv("titanic.csv")

passengerCountsBySex = df.groupBy("Pclass", "Sex").agg(count("*").alias("count"))

passengerCountsBySex.show()


+------+------+-----+
|Pclass|   Sex|count|
+------+------+-----+
|     2|female|   76|
|     3|  male|  347|
|     1|  male|  122|
|     3|female|  144|
|     1|female|   94|
|     2|  male|  108|
+------+------+-----+



In [5]:
# Determining the number of passengers between survivors and non-survivors

from pyspark.sql.functions import count

df = spark.read.option("header", "true").option("inferSchema", "true").csv("titanic.csv")

survivorsCounts = df.groupBy("Survived").agg(count("*").alias("count"))

survivorsCounts.show()


+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [6]:
# Average age of passengers by gender and by class:

from pyspark.sql.functions import avg

df = spark.read.option("header", "true").option("inferSchema", "true").csv("titanic.csv")

avgAgeBySexAndClass = df.groupBy("Pclass", "Sex").agg(avg("Age").alias("avg_age"))

avgAgeBySexAndClass.show()


+------+------+------------------+
|Pclass|   Sex|           avg_age|
+------+------+------------------+
|     2|female|28.722972972972972|
|     3|  male|26.507588932806325|
|     1|  male| 41.28138613861386|
|     3|female|             21.75|
|     1|female| 34.61176470588235|
|     2|  male| 30.74070707070707|
+------+------+------------------+



In [7]:
# Average age of passengers by survival

from pyspark.sql.functions import avg

df = spark.read.option("header", "true").option("inferSchema", "true").csv("titanic.csv")

avgAgeBySurvival = df.groupBy("Survived").agg(avg("Age").alias("avg_age"))

avgAgeBySurvival.show()


+--------+------------------+
|Survived|           avg_age|
+--------+------------------+
|       1|28.343689655172415|
|       0| 30.62617924528302|
+--------+------------------+



In [8]:
# Average age of first class passengers

from pyspark.sql.functions import avg

df = spark.read.option("header", "true").option("inferSchema", "true").csv("titanic.csv")

avgAgeFirstClass = df.filter(df.Pclass == 1).agg(avg("Age").alias("avg_age"))

avgAgeFirstClass.show()


+------------------+
|           avg_age|
+------------------+
|38.233440860215055|
+------------------+



In [9]:
# Average age of passengers by class and survival status

from pyspark.sql.functions import avg

df = spark.read.option("header", "true").option("inferSchema", "true").csv("titanic.csv")

avgAgeByClassAndSurvival = df.groupBy("Pclass", "Survived").agg(avg("Age").alias("avg_age"))

avgAgeByClassAndSurvival.show()


+------+--------+------------------+
|Pclass|Survived|           avg_age|
+------+--------+------------------+
|     1|       0|        43.6953125|
|     3|       1|20.646117647058823|
|     1|       1| 35.36819672131148|
|     2|       1| 25.90156626506024|
|     2|       0|33.544444444444444|
|     3|       0|26.555555555555557|
+------+--------+------------------+



## Exercises to solve:

In [10]:
# Determine the age of the oldest and youngest passengers.
from pyspark.sql.functions import avg, min, max

df = spark.read.option("header", "true").option("inferSchema", "true").csv("titanic.csv")

age_aggregation = df.agg(min("Age"),max("Age"))

age_aggregation.show()


+--------+--------+
|min(Age)|max(Age)|
+--------+--------+
|    0.42|    80.0|
+--------+--------+



Expected output:

```
+--------+--------+  
|min(Age)|max(Age)|  
+--------+--------+  
|    0.42|    80.0|  
+--------+--------+    
```


In [11]:
# Determining the number of women and men aged 30 and over in first class
df = spark.read.option("header", "true").option("inferSchema", "true").csv("titanic.csv")

females_cnt = df.filter("Pclass = 1 and Age > 30 and Sex == 'female'" ).count()
males_cnt = df.filter("Pclass = 1 and Age > 30 and Sex == 'male'" ).count()

print(f'First class, number of women over 30: {females_cnt}')
print(f'First class, age 30 and over: {males_cnt}')

First class, number of women over 30: 50
First class, age 30 and over: 75


Expected output:

```
First class, number of women over 30: 50
First class, age 30 and over: 75
```


In [12]:
# Passenger survival rates by cabin.
from pyspark.sql.functions import avg

df = spark.read.option("header", "true").option("inferSchema", "true").csv("titanic.csv")
cabin_survivors = df.groupBy("Cabin").agg(avg("Survived").alias("SurvivalRate"))

cabin_survivors.show()

+-------+------------------+
|  Cabin|      SurvivalRate|
+-------+------------------+
|    A23|               1.0|
|    B79|               1.0|
|    E44|               0.5|
|  F E69|               1.0|
|    D28|               1.0|
|    C78|               0.5|
|    C95|               0.0|
|  F G73|               0.0|
|B58 B60|               0.5|
|     D7|               1.0|
|   C128|               0.0|
|    B39|               1.0|
|    B22|               0.5|
|   C110|               0.0|
|    D21|               1.0|
|     F2|0.6666666666666666|
|    B30|               0.0|
|   C104|               1.0|
|    B50|               1.0|
|     A6|               1.0|
+-------+------------------+
only showing top 20 rows



Expected output:

```
+-------+------------------+
|  Cabin|      SurvivalRate|
+-------+------------------+
|    A23|               1.0|
|    B79|               1.0|
|    E44|               0.5|
|  F E69|               1.0|
|    D28|               1.0|
|    C78|               0.5|
|    C95|               0.0|
|  F G73|               0.0|
|B58 B60|               0.5|
|     D7|               1.0|
|   C128|               0.0|
|    B39|               1.0|
|    B22|               0.5|
|   C110|               0.0|
|    D21|               1.0|
|     F2|0.6666666666666666|
|    B30|               0.0|
|   C104|               1.0|
|    B50|               1.0|
|     A6|               1.0|
+-------+------------------+
only showing top 20 rows
```


In [13]:
# Passenger survival based on the place of embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
from pyspark.sql.functions import avg
df = spark.read.option("header", "true").option("inferSchema", "true").csv("titanic.csv")

embarked_survivors = df.groupBy("Embarked").agg(avg("Survived") .alias("SurvivalRate"))
embarked_survivors.show()

+--------+-------------------+
|Embarked|       SurvivalRate|
+--------+-------------------+
|       Q|0.38961038961038963|
|    null|                1.0|
|       C| 0.5535714285714286|
|       S|0.33695652173913043|
+--------+-------------------+



Expected output:

```
+--------+-------------------+
|Embarked|       SurvivalRate|
+--------+-------------------+
|       Q|0.38961038961038963|
|    null|                1.0|
|       C| 0.5535714285714286|
|       S|0.33695652173913043|
+--------+-------------------+
```


In [14]:
# The most common age group among passengers is 10 years.Scroll down. 0, 10, 20, etc..
from pyspark.sql.functions import round, expr, count

df = spark.read.option("header", "true").option("inferSchema", "true").csv("titanic.csv")
rounded_age = df.withColumn('rounded_age', expr('round(age, -1)')).groupBy('rounded_age').agg(count('*').alias('count')).orderBy("count", ascending=False)

rounded_age.show()


+-----------+-----+
|rounded_age|count|
+-----------+-----+
|       30.0|  201|
|       20.0|  200|
|       null|  177|
|       40.0|  120|
|       50.0|   73|
|        0.0|   40|
|       10.0|   38|
|       60.0|   31|
|       70.0|   10|
|       80.0|    1|
+-----------+-----+



Expected output:

```
+-----------+-----+
|Age_rounded|count|
+-----------+-----+
|       30.0|  201|
|       20.0|  200|
|       40.0|  120|
|       50.0|   73|
|        0.0|   40|
|       10.0|   38|
|       60.0|   31|
|       70.0|   10|
|       80.0|    1|
|       null|    0|
+-----------+-----+
```


In [15]:
# The survival rate of passengers in the most common age group.
from pyspark.sql.functions import avg, round, expr, count

df = spark.read.option("header", "true").option("inferSchema", "true").csv("titanic.csv")
rounded_age_survival = df.withColumn('rounded_age', expr('round(age, -1)')).groupBy('rounded_age').agg(count('*').alias('count'), avg('Survived')).orderBy("count", ascending=False)

rounded_age_survival.show()

+-----------+-----+-------------------+
|rounded_age|count|      avg(Survived)|
+-----------+-----+-------------------+
|       30.0|  201| 0.3880597014925373|
|       20.0|  200|              0.365|
|       null|  177| 0.2937853107344633|
|       40.0|  120|              0.425|
|       50.0|   73|  0.410958904109589|
|        0.0|   40|              0.675|
|       10.0|   38|0.47368421052631576|
|       60.0|   31| 0.3870967741935484|
|       70.0|   10|                0.0|
|       80.0|    1|                1.0|
+-----------+-----+-------------------+



Expected output:

```
+-----------+-----+-------------------+
|Age_rounded|count|        AvgSurvived|
+-----------+-----+-------------------+
|       30.0|  201| 0.3880597014925373|
|       20.0|  200|              0.365|
|       40.0|  120|              0.425|
|       50.0|   73|  0.410958904109589|
|        0.0|   40|              0.675|
|       10.0|   38|0.47368421052631576|
|       60.0|   31| 0.3870967741935484|
|       70.0|   10|                0.0|
|       80.0|    1|                1.0|
|       null|    0| 0.2937853107344633|
+-----------+-----+-------------------+
```


In [16]:
# Average age and class of passengers by survival.
from pyspark.sql.functions import avg

df = spark.read.option("header", "true").option("inferSchema", "true").csv("titanic.csv")

survival_age = df.groupBy("Survived").agg(avg("Age"), avg("Pclass"))

survival_age.show()

+--------+------------------+------------------+
|Survived|          avg(Age)|       avg(Pclass)|
+--------+------------------+------------------+
|       1|28.343689655172415|1.9502923976608186|
|       0| 30.62617924528302|2.5318761384335153|
+--------+------------------+------------------+



Expected output:

```
+--------+------------------+------------------+
|Survived|          avg(Age)|       avg(Pclass)|
+--------+------------------+------------------+
|       1|28.343689655172415|1.9502923976608186|
|       0| 30.62617924528302|2.5318761384335153|
+--------+------------------+------------------+
```


In [17]:
# Total number of passengers, by class and gender.
from pyspark.sql.functions import count


df = spark.read.option("header", "true").option("inferSchema", "true").csv("titanic.csv")

class_sex = df.groupBy("Pclass", "Sex").agg(count("PassengerId"))

class_sex.show()


+------+------+------------------+
|Pclass|   Sex|count(PassengerId)|
+------+------+------------------+
|     2|female|                76|
|     3|  male|               347|
|     1|  male|               122|
|     3|female|               144|
|     1|female|                94|
|     2|  male|               108|
+------+------+------------------+



Expected output:

```
+------+------+------------------+
|Pclass|   Sex|count(PassengerId)|
+------+------+------------------+
|     2|female|                76|
|     3|  male|               347|
|     1|  male|               122|
|     3|female|               144|
|     1|female|                94|
|     2|  male|               108|
+------+------+------------------+
```


In [18]:
# Calculating the survival rate among passengers whose cabin is unknown.
from pyspark.sql.functions import avg

df = spark.read.option("header", "true").option("inferSchema", "true").csv("titanic.csv")

no_cabin = df.filter("Cabin is NULL").agg(avg("Survived"))

no_cabin.show()


+-------------------+
|      avg(Survived)|
+-------------------+
|0.29985443959243085|
+-------------------+



Expected output:

```
+-------------------+
|      avg(Survived)|
+-------------------+
|0.29985443959243085|
+-------------------+
```


In [19]:
# Total number of passengers with no known cabin, by class and survival status.
df = spark.read.option("header", "true").option("inferSchema", "true").csv("titanic.csv")

class_survived = df.filter("Cabin is NULL").groupBy("Pclass", "Survived").agg(count("PassengerId"))

class_survived.show()



+------+--------+------------------+
|Pclass|Survived|count(PassengerId)|
+------+--------+------------------+
|     1|       0|                21|
|     3|       1|               113|
|     1|       1|                19|
|     2|       1|                74|
|     2|       0|                94|
|     3|       0|               366|
+------+--------+------------------+



Expected output:

```
+------+--------+------------------+
|PClass|Survived|count(PassengerId)|
+------+--------+------------------+
|     1|       0|                21|
|     3|       1|               113|
|     1|       1|                19|
|     2|       1|                74|
|     2|       0|                94|
|     3|       0|               366|
+------+--------+------------------+
```
